In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [85]:
from numba import njit

@njit
def findAngles(a, b, c):
        # applied cosine rule
        A = np.arccos((b * b + c * c - a * a) / (2 * b * c))
        B = np.arccos((a * a + c * c - b * b) / (2 * a * c))
        C = np.arccos((b * b + a * a - c * c) / (2 * b * a))
        # convert into degrees
        A, B, C = np.rad2deg(A), np.rad2deg(B), np.rad2deg(C)
        return A, B, C


@njit
def compute_K_vet(triplet):
        a, b, c = compute_sides(triplet)
        A, B, C = findAngles(a, b, c)
        K_vet = np.array([A, B, C])
        if K_vet is not None:
            return K_vet

@njit
def compute_sides(triplet):
        a = np.linalg.norm(triplet[0][0:2] - triplet[1][0:2])
        b = np.linalg.norm(triplet[1][0:2] - triplet[2][0:2])
        c = np.linalg.norm(triplet[2][0:2] - triplet[0][0:2])
        return a, b, c


def find_all_triplets(craters):

    def Hstack(K_v, i,j,k,x1,y1,r1,x2,y2,r2,x3,y3,r3):
        A = np.zeros(15)
        A[0],A[1],A[2] = K_v[0],K_v[1],K_v[2]
        A[3],A[4],A[5] = i,j,k
        A[6],A[7],A[8] = x1,y1,r1
        A[9],A[10],A[11] = x2,y2,r2
        A[12],A[13],A[14] = x3,y3,r3
        return A


    def concat(a,b,c):
        A = np.zeros((3,3))
        A[0] = a
        A[1] = b
        A[2] = c
        return A

    # Input: np.array craters
    # Output: all triplets
    N = craters.shape[0]
    ender = N*N*N
    K = np.zeros((ender,15))
    lister = 0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if (i!=j) & (j!=k):
                    a =craters[i]
                    b =craters[j]
                    c =craters[k]
                    triplet = concat(a,b,c)
                    x1,y1,r1 = a[0],a[1],a[2]
                    x2,y2,r2 = b[0],b[1],b[2]
                    x3,y3,r3 = c[0],c[1],c[2]
                    
                    C = np.zeros(2) # centroid
                    C[0] = (x1+x2+x3)/3
                    C[1] = (y1+y2+y3)/3

                    P1,P2,P3 = np.zeros(2), np.zeros(2), np.zeros(2)
                    P1[0]=x1
                    P1[1]=y1
                    P2[0]=x2
                    P2[1]=y2
                    P3[0]=x3
                    P3[1]=y3

                    d1,d2,d3 = np.linalg.norm(P1-C), np.linalg.norm(P2-C),np.linalg.norm(P3-C)
                    d_i,d_j,d_k = d1/r1, d2/r2, d3/r3


                    try:
                        K_v = compute_K_vet(triplet)
                        K[lister] = Hstack(K_v, d_i,d_j,d_k,x1,y1,r1,x2,y2,r2,x3,y3,r3)
                    except ZeroDivisionError: pass

                lister+=1
    return K[ np.all(K !=0, axis=1) ]

In [3]:
# LOAD ALL IMAGES:
dt = 10
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [143]:
idx = 40
tol = 1.5
# IMG1:
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
# IMG2:
img2=cv2.imread(dict[str(idx+2)])
craters_det2 = detect(img2)

# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
df_craters_det2 = sort_mat(craters_det2)

H1 = find_all_triplets(craters_det1)
H2 = find_all_triplets(craters_det2)
QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
QUERY2= pd.DataFrame(H2, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
# # remove wrong triplet:
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
########################################################################################################################################################
# SEARCH
########################################################################################################################################################
found = False
for idx in range(QUERY1.shape[0]):
    Angle1_Q1 = QUERY1.Angle1[idx]
    Angle2_Q1 = QUERY1.Angle2[idx]
    Angle3_Q1 = QUERY1.Angle3[idx]
    d1_Q1 = QUERY1.des1[idx]
    d2_Q1 = QUERY1.des2[idx]
    d3_Q1 = QUERY1.des3[idx]

    SEARCH = QUERY2[ ((abs(QUERY2.Angle1 - Angle1_Q1) < tol) & (abs(QUERY2.Angle2 - Angle2_Q1) < tol) & (abs(QUERY2.Angle3 - Angle3_Q1) < tol))   \
                      |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol)) \
                      |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol))]

    SEARCH = SEARCH[ ((abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des1- d3_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des1- d1_Q1) < tol))\
                     | ((abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des1- d2_Q1) < tol))]
    if SEARCH.shape[0] != 0:
        print('FOUND!')
        save = [idx, SEARCH.iloc[0].name]
        found = True
        break
if found:
    a = QUERY1.iloc[save[0]]
    b = QUERY2.iloc[save[1]]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img1.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)

    plt.subplot(122)
    cp2 = img2.copy()
    IMG2 =  img_plus_crts(cp2, crts2, color="red")
    plt.imshow(IMG2)
    plt.show()
else:
    print("Match can't find any triplets.")

FOUND!


174

In [44]:
SEARCH

,Angle1,Angle2,Angle3,ctr1,ctr2,ctr3,x1,y1,r1,x2,y2,r2,x3,y3,r3
577,152.062971,15.937871,11.999158,6.0,3.0,10.0,69.5,641.0,16.25,710.5,128.0,17.75,280.5,344.0,18.75


In [47]:
QUERY1.iloc[idx:idx+1]

,Angle1,Angle2,Angle3,ctr1,ctr2,ctr3,x1,y1,r1,x2,y2,r2,x3,y3,r3
0,13.708194,151.282071,15.009736,1.0,2.0,4.0,339.0,493.5,16.25,317.0,795.5,15.25,201.5,192.5,17.0


In [ ]:
# # Searching:
# count,d, found = 0, 10, False
# # for i in QUERY1.iloc:
# for idx in range(QUERY1.shape[0]):
#     if found:
#         break

#     i = QUERY1.iloc[idx]
#     count+=1
#     printProgressBar(count, QUERY1.shape[0], printEnd='')
    
#     for idy in range(QUERY2.shape[0]):
#         j = QUERY2.iloc[idy]
#         a1,a2,a3 = i.Angle1,i.Angle2,i.Angle3
#         b1,b2,b3 = j.Angle1,j.Angle2,j.Angle3
#         x1,x2,x3 = i.r1,i.r2,i.r3
#         y1,y2,y3 = j.r1,j.r2,j.r3
# #       switch
#         d1 = abs(a1-b1)+abs(a2-b2)+abs(a3-b3)
#         d2 = abs(a1-b2)+abs(a2-b3)+abs(a3-b1)
#         d3 = abs(a1-b3)+abs(a2-b1)+abs(a3-b2)
# #       switch
#         r1 = abs(x1-y1)+abs(x2-y2)+abs(x3-y3)
#         r2 = abs(x1-y2)+abs(x2-y3)+abs(x3-y1)
#         r3 = abs(x1-y3)+abs(x2-y1)+abs(x3-y2)

#         d = min(d1,d2,d3)
#         r = min(r1,r2,r3)
#         if (d < 2) & (r < 5):
#             print('\nFOUNDED')
#             save = [idx, idy]
#             print(save)
#             print(d,r)
#             found = True
#             break
# print('\n')
# if found:
#     a = QUERY1.iloc[save[0]]
#     b = QUERY2.iloc[save[1]]
#     crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
#     crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

#     plt.figure(dpi=200)
#     plt.subplot(121)
#     cp1 = img1.copy()
#     IMG1 =  img_plus_crts(cp1, crts1, color="red")
#     plt.imshow(IMG1)

#     plt.subplot(122)
#     cp2 = img2.copy()
#     IMG2 =  img_plus_crts(cp2, crts2, color="red")
#     plt.imshow(IMG2)
#     plt.show()
# else:
#     print("Match can't find any triplets.")

In [ ]:
# def find_query(df_craters):
#     STORED = []
#     c=0
#     M=df_craters.shape[0]
#     QUERY = np.zeros((M**2,15))
#     for i in range(M):
#         for j in range(M):
#             if i != j:
#                 triplet, HP, PICKS = find_triplet(df_craters, idx_a = i, idx_b=j)
#                 Names = [triplet[0].name,triplet[1].name,triplet[2].name]
#                 names = set(Names)
#                 if names not in STORED:
#                     STORED.append(names)
#                     K_v = compute_K_vet(triplet)
#                     c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
#                     c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
#                     c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
#                     QUERY[c] = np.hstack([K_v, Names,c1,c2,c3])
#                     c+=1
#                 else: continue

#         printProgressBar(i+1, df_craters.shape[0], printEnd='')
#     QUERY = pd.DataFrame(QUERY, columns=['Angle1','Angle2','Angle3','ctr1','ctr2','ctr3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
#     return QUERY

In [ ]:
# # Searching:
# count,d, found = 0, 10, False
# # for i in QUERY1.iloc:
# for idx in range(QUERY1.shape[0]):
#     if found:
#         break

#     i = QUERY1.iloc[idx]
#     count+=1
#     printProgressBar(count, QUERY1.shape[0], printEnd='')
    
#     for idy in range(QUERY2.shape[0]):
#         j = QUERY2.iloc[idy]
#         a1,a2,a3 = i.Angle1,i.Angle2,i.Angle3
#         b1,b2,b3 = j.Angle1,j.Angle2,j.Angle3
#         x1,x2,x3 = i.r1,i.r2,i.r3
#         y1,y2,y3 = j.r1,j.r2,j.r3
# #       switch
#         d1 = abs(a1-b1)+abs(a2-b2)+abs(a3-b3)
#         d2 = abs(a1-b2)+abs(a2-b3)+abs(a3-b1)
#         d3 = abs(a1-b3)+abs(a2-b1)+abs(a3-b2)
# #       switch
#         r1 = abs(x1-y1)+abs(x2-y2)+abs(x3-y3)
#         r2 = abs(x1-y2)+abs(x2-y3)+abs(x3-y1)
#         r3 = abs(x1-y3)+abs(x2-y1)+abs(x3-y2)

#         d = min(d1,d2,d3)
#         r = min(r1,r2,r3)
#         if (d < 2) & (r < 5):
#             print('\nFOUNDED')
#             save = [idx, idy]
#             print(save)
#             print(d,r)
#             found = True
#             break
# print('\n')
# if found:
#     a = QUERY1.iloc[save[0]]
#     b = QUERY2.iloc[save[1]]
#     crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
#     crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

#     plt.figure(dpi=200)
#     plt.subplot(121)
#     cp1 = img1.copy()
#     IMG1 =  img_plus_crts(cp1, crts1, color="red")
#     plt.imshow(IMG1)

#     plt.subplot(122)
#     cp2 = img2.copy()
#     IMG2 =  img_plus_crts(cp2, crts2, color="red")
#     plt.imshow(IMG2)
#     plt.show()
# else:
#     print("Match can't find any triplets.")

In [ ]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [ ]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [ ]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [ ]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [ ]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)